## Importing libraries

In [2]:
import sys
gp = '/home/jupyter-gabriel/'
sys.path.append(gp + "projects/suyana")
from scripts.utils.functions import *

import requests

# Processed canton data

In [3]:
import os
path = gp + "projects/suyana/data/capacity/"
files = os.listdir(path)
files = [f for f in files if f.endswith(".nc")]

id_canton = [files[i].split("_")[1].split(".")[0] for i in range(len(files))]

IndexError: list index out of range

In [129]:
list_dates = []
list_swc = []
list_id = []
data = pd.DataFrame()
for file in files:
    id_canton = file.split("_")[1].split(".")[0]
    netcdf_file = xr.open_dataset(path + file)
    
    data_aux = pd.DataFrame({
        "date": netcdf_file.time.values,
        "swc": netcdf_file.swc_adjusted.values
    })
    data_aux["idCanton"] = id_canton

    data = pd.concat([data, data_aux], axis=0)

## Exploratory Data Analysis

In [62]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [140]:
netcdf_file.variables

Frozen({'swc_raw': <xarray.Variable (time: 8176)> Size: 65kB
array([     nan, 0.308279, 0.2454  , ..., 0.320967, 0.319731,      nan],
      shape=(8176,)), 'week': <xarray.Variable (time: 8176)> Size: 65kB
[8176 values with dtype=float64], 'swc_adjusted': <xarray.Variable (time: 8176)> Size: 65kB
array([     nan, 0.350339, 0.269238, ..., 0.320967, 0.319731,      nan],
      shape=(8176,)), 'time': <xarray.IndexVariable 'time' (time: 8176)> Size: 65kB
array(['2002-06-15T00:00:00.000000000', '2002-06-16T00:00:00.000000000',
       '2002-06-17T00:00:00.000000000', ..., '2024-10-29T00:00:00.000000000',
       '2024-10-30T00:00:00.000000000', '2024-10-31T00:00:00.000000000'],
      shape=(8176,), dtype='datetime64[ns]')})

In [64]:
data_summary = data.groupby(["idCanton", "year"]).size().reset_index()

In [65]:
## CANTIDAD DE OBSERVACIONES POR AÑO
data_summary.pivot(index="idCanton", columns="year", values=0)

year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
idCanton,,,,,,,,,,,,,,,,,,,,,,,
43,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
45,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
49,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
50,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
51,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
52,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
53,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
54,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
55,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305


In [70]:
## DUPLICADOS
data[data.year==2012].groupby(["date", "idCanton"]).agg(n=("idCanton","count")).reset_index().query('n>1')

,date,idCanton,n
3485,2012-07-24,43,2
3486,2012-07-24,45,2
3487,2012-07-24,49,2
3488,2012-07-24,50,2
3489,2012-07-24,51,2
3490,2012-07-24,52,2
3491,2012-07-24,53,2
3492,2012-07-24,54,2
3493,2012-07-24,55,2
3494,2012-07-24,56,2


In [74]:
n_anios = len(data['year'].unique())
min_year = min(data['year'])
max_year = max(data['year'])
print(f'Cantidad de años: {nAnios} desde {min_year} hasta {max_year}')
n_dates = len(data['date'].unique())
min_date = min(data['date'])
max_date = max(data['date'])
print(f'Cantidad de días: {n_dates} desde {min_date} hasta {max_date}')

Cantidad de años: 23 desde 2002 hasta 2024
Cantidad de días: 8175 desde 2002-06-15 00:00:00 hasta 2024-10-31 00:00:00


# Get canton id for yields panel data

In [3]:
# Data from gpkg cantones bolivia
df_cantones = gpd.read_file(gp + 'projects/suyana/data/drought/sub_AOIs_priority.gpkg')

ERROR 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed


In [4]:
# data_folder = Path("../Data/data_output_rendimiento")
# filename = "df_rendimiento_flagged_ONI.csv"
data_folder = gp + "projects/suyana/data/drought/"
filename = "yields_panel.csv"
df_yields = pd.read_csv(data_folder + filename)
df_yields.drop(['flag','notes', 'Unnamed: 10'], axis=1, inplace = True)

In [5]:
df_aux = df_yields[['longitude', 'latitude']].drop_duplicates()
df_aux['coords'] = list(zip(df_aux['longitude'], df_aux['latitude']))

df_coords = gpd.GeoDataFrame({
    'longitude': list(df_aux['longitude']),
    'latitude': list(df_aux['latitude']),
    'geometry': [Point(p) for p in df_aux['coords']]
}, crs = df_cantones.crs)

df_join = df_coords.sjoin(
    df_cantones,
    how = 'left',
    predicate = 'within'
)

In [6]:
df_yields_final = df_yields.merge(
    df_join[['longitude', 'latitude', 'id_canton', 'canton_original', 'provincia', 'municipio', 'departamen']],
    how = 'left',
    on = ['longitude', 'latitude'],
)

In [7]:
df_yields_final.id_canton.unique()

array([189,  56,  52,  49,  57, 195,  43,  54, 209])

## Clean data from yields panel

In [8]:
def group_categories_MT(df_orig, group_categories):
    """
    Groups the categories of a categorical variable in a DataFrame.

    Parameters:
        df_orig (pd.DataFrame): DataFrame containing the client variables.
        group_categories (dict): Dictionary that contains the categories to be grouped.
                                 The keys are the column names, and the values are dictionaries
                                 where the keys are the new categories and the values are lists
                                 of the original categories to be grouped under the new category.

    Returns:
        pd.DataFrame: Modified DataFrame with grouped category columns.
    """
    df = df_orig.copy()
    
    features_group = group_categories["categories"]

    for column in features_group:
        for new_category, original_categories in features_group[column].items():
            df[column] = df[column].replace(original_categories, new_category)

    return df

In [9]:
def modify_categories(
    df_orig: pd.DataFrame(), params_modificacion_categorias
) -> pd.DataFrame():
    """Modify categories in the dataset based on specified values.

    Args:
        df_orig (pd.DataFrame): The complete dataset with all variables.
        params_modificacion_categorias (dict): A dictionary containing the column and the desired categories for that column.
            Other categories not specified will be replaced with 'OTRO'.

    Returns:
        pd.DataFrame: The modified dataset with updated categories.

    Example:
        >>> df = modify_categories(df, params_modificacion_categorias)
    """

    df = df_orig.copy()

    # Parameters
    variables_valores = params_modificacion_categorias["variables_valores"]

    for col in variables_valores:
        print(col)
        df[col] = df[col].apply(
            lambda x: x if x in variables_valores[col] else "OTRO"
        )

    return df


In [10]:
def treat_nans(df_orig, params):
    """
    Treats missing values in the DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing the client variables.
        params (dict): Dictionary containing the parameters to treat missing values.

    Returns:
        pd.DataFrame: Modified DataFrame with treated missing values.
    """
    df = df_orig.copy()
    # Parameters
    missing_vars = params["missing_vars"]
    method = params["method"]
    group_vars = params["group_vars"]

    if set(group_vars).intersection(set(missing_vars)) != set():
        print("Error: group_vars and missing_vars should not have common variables.")
        return
    # Treat missing values
    if method == "remove":
        df = df.dropna(subset=missing_vars)
    elif method == "impute":
        for var in missing_vars:
            if df[var].dtype == "object":
                df[var] = df[var].fillna("MISSING")
            elif len(group_vars) == 0:
                df[var] = df[var].fillna(df[var].mean())
            else:
                df[var] = df.groupby(group_vars)[var].transform(
                    lambda x: x.fillna(x.mean())
                )

    return df

In [11]:
def apply_log_to_var(df_orig, var, fill_value="mean"):
    """
    Apply the natural logarithm to the specified variables in the DataFrame.

    Parameters:
        df_orig (pd.DataFrame): DataFrame containing the client variables.
        params (dict): Dictionary containing the parameters to apply the natural logarithm.

    Returns:
        pd.DataFrame: Modified DataFrame with the natural logarithm applied to the specified variables.
    """
    df = df_orig.copy()
       
    # Apply the natural logarithm
    min_value = df.loc[df[var] > 0, var].min()
    df[var] = np.where(df[var] <= 0, min_value*0.9, df[var])

    df["log_"+var] = np.log(df[var])
    df["log_"+var] = np.where(
        df["log_"+var].isin([np.inf, -np.inf]), 
        np.nan, 
        df["log_"+var]
    )
    if fill_value == "mean":
        df["log_"+var] = df["log_"+var].fillna(df["log_"+var].mean())
    else:
        df["log_"+var] = df["log_"+var].fillna(fill_value)

    return df

In [12]:
params_group_categ = {
    'categories': {
        'culture': {
            #'GIRASOL': ['Girasol'],
            'MAIZ': ['MAIZ', 'MAIZ ENSILAJE'],
            'TRIGO': ['TRIGO'],
            'SOYA': ['SOYA'],
            'SORGO': ['SORGO', 'SORGO DOBLE PROPOSITO', 'SORGO DOBLE PROPOSITO ENSILAJE', 'SORGO FORRAJERO'],
        }
    }
}
params_modif_categ = {
    'variables_valores': {
        'culture': ['MAIZ', 'TRIGO', 'SOYA', 'SORGO']
    }
}
params_nans = {
    'missing_vars': ['rend_ton_ha'], #, 'ndvi', 'ndpi', 'pp', 'vhi', 'anomaly_sum', 'swc_sum'
    'method': 'impute',
    'group_vars': []
}

In [13]:
df_yields_final['culture_orig'] = df_yields_final['culture']
df_yields_final = group_categories_MT(df_yields_final, params_group_categ)
df_yields_final = modify_categories(df_yields_final, params_modif_categ)
df_yields_final = treat_nans(df_yields_final, params_nans)

culture


In [14]:
df_yields_final['culture'].value_counts()

culture
SOYA     433
SORGO    116
TRIGO    101
MAIZ      59
OTRO      51
Name: count, dtype: int64

In [15]:
df_yields_final.to_csv(gp + 'projects/suyana/data/drought/yields_panel_id.csv', index=False)

In [9]:
df_yields_final = pd.read_csv(gp+'projects/suyana/data/drought/yields_panel_id.csv')

# Final yields_panel data

In [138]:
df_yields_final = df_yields_final.loc[~(df_yields_final.seed_date.isna()),:].copy()

In [140]:
df_yields_final['harvest_date'] = pd.to_datetime(df_yields_final['harvest_date'])
df_yields_final['seed_date'] = pd.to_datetime(df_yields_final['seed_date'])
df_yields_final['flowering_date'] = pd.to_datetime(df_yields_final['flowering_date'])

In [150]:
df_yields_final.groupby([df_yields_final.seed_date.dt.month, df_yields_final.harvest_date.dt.month]).agg(n = ('id_lot', 'count')).reset_index().tail(20)

,seed_date,harvest_date,n
25,8,8.0,1
26,8,10.0,1
27,8,11.0,9
28,8,12.0,10
29,9,1.0,1
30,9,11.0,1
31,11,1.0,1
32,11,2.0,7
33,11,3.0,129
34,11,4.0,39


In [144]:
df_yields_final['cropYear'] = (df_yields_final.seed_date - pd.DateOffset(months=3)).dt.year
df_yields_final['cropCycle'] = np.where(
    #(df_final['month'].isin([11,12,1])) | ((df_final['month']==2) & (df_final['day']<=15)),
    ~(df_yields_final.seed_date.dt.month.isin([13])),
    df_yields_final['cropYear'].astype(str),
    df_yields_final.seed_date.dt.year.astype(str) + '_'
)
df_yields_final['flagSummer'] = np.where(df_yields_final.seed_date.dt.month.isin([11,12,1,2,3,4]), 1, 0)

In [145]:
df_yields_final.cropCycle.value_counts()

cropCycle
2014    167
2015    157
2016    154
2017    131
2013     75
2018     32
2012      3
Name: count, dtype: int64

In [68]:
df_yields_final.harvest_date.dt.month.value_counts()

harvest_date
3.0     168
8.0     156
4.0     145
9.0      67
11.0     60
10.0     39
7.0      35
12.0     16
6.0      12
2.0       7
1.0       3
5.0       2
Name: count, dtype: int64

In [146]:
# Get the nearest anomaly crop cycle
df_anomalies = df_final.groupby(['lon', 'lat', 'cropCycle', 'flagSummer']).agg(
    accumulatedAnomaly = ('accumulatedAnomaly', 'max'),
    #accumulatedAnomaly_2 = ('negativeAnomaly', 'sum'),
).reset_index()
df_anomalies

,lon,lat,cropCycle,flagSummer,accumulatedAnomaly
0,-63.48737,-16.36514,2002,0,3.480032
1,-63.48737,-16.36514,2002,1,1.623479
2,-63.48737,-16.36514,2003,0,3.350986
3,-63.48737,-16.36514,2003,1,4.073963
4,-63.48737,-16.36514,2004,0,1.572667
...,...,...,...,...,...
4810,-61.93275,-17.08809,2022,0,3.254927
4811,-61.93275,-17.08809,2022,1,2.154343
4812,-61.93275,-17.08809,2023,0,1.942862
4813,-61.93275,-17.08809,2023,1,2.579326


In [147]:
df_yields_join = df_yields_final.merge(
    df_anomalies,
    how = 'left',
    left_on = ['longitude', 'latitude', 'cropCycle', 'flagSummer'],
    right_on = ['lon', 'lat', 'cropCycle', 'flagSummer']
).drop(['lon', 'lat'], axis=1)
df_yields_join[['longitude', 'latitude', 'cropCycle', 'flagSummer', 'accumulatedAnomaly']].head(10)

,longitude,latitude,cropCycle,flagSummer,accumulatedAnomaly
0,-63.48191,-16.36096,2016,0,5.631754
1,-62.35004,-17.05833,2012,1,2.179553
2,-62.33751,-17.07043,2012,1,2.192454
3,-62.40913,-16.92304,2012,1,1.708325
4,-62.40973,-16.92482,2013,1,1.366242
5,-62.34688,-17.09826,2013,1,1.964897
6,-62.36080,-17.08261,2013,1,1.892275
7,-62.28137,-17.30583,2013,1,2.883126
8,-62.41933,-16.92666,2013,1,1.343708
9,-62.41945,-16.92877,2013,1,1.345889


In [148]:
df_yields_join.to_csv(gp + 'projects/suyana/data/drought/yields_panel_final.csv', index=False)
#df_yields_join

## CDF Matching function example

In [166]:
x = np.array([5,4,4,3,2,np.nan])
y = np.array([2,1,1,4,np.nan, np.nan])

x_valid = x[~np.isnan(x)]
y_valid = y[~np.isnan(y)]

x_cdf = np.sort(x_valid)
y_cdf = np.sort(y_valid)

x_indices = np.searchsorted(x_cdf, x, side="right")
y_quantiles = x_indices / len(x_cdf)  # Quantiles of x in the valid x range
y_indices = np.floor(y_quantiles * (len(y_cdf)-0)).astype(
    int
)  # Corresponding indices in y

adjusted_x = np.take(y_cdf, y_indices, mode="clip")

In [167]:
adjusted_x

array([4., 4., 4., 1., 1., 4.])

In [168]:
y_indices

array([4, 3, 3, 1, 0, 4])

In [169]:
y_cdf

array([1., 1., 2., 4.])

In [170]:
y_quantiles

array([1. , 0.8, 0.8, 0.4, 0.2, 1. ])

# Toy data for xarray

In [103]:
ds_toy = xr.Dataset(
    data_vars = dict(
        temp = (['lon', 'lat','time'], np.ones((5,5,2))),
    ),
    coords = dict(
        lon = np.arange(1,6),
        lat = np.arange(1,6),
        time = np.arange(1,3)
    )
)
ds_toy

<xarray.Dataset> Size: 496B
Dimensions:  (lon: 5, lat: 5, time: 2)
Coordinates:
  * lon      (lon) int64 40B 1 2 3 4 5
  * lat      (lat) int64 40B 1 2 3 4 5
  * time     (time) int64 16B 1 2
Data variables:
    temp     (lon, lat, time) float64 400B 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0

In [104]:
ds_toy2 = ds_toy.coarsen({'lon':3, 'lat': 3}, boundary='pad').sum()

ds_toy2.to_dataframe().reset_index()

,lon,lat,time,temp
0,2.0,2.0,1,9.0
1,2.0,2.0,2,9.0
2,2.0,4.5,1,6.0
3,2.0,4.5,2,6.0
4,4.5,2.0,1,6.0
5,4.5,2.0,2,6.0
6,4.5,4.5,1,4.0
7,4.5,4.5,2,4.0


In [45]:
ds_toy.coarsen(lon=len(ds_toy.lon)-len(ds_toy.lon)//2, lat=len(ds_toy.lat)-len(ds_toy.lat)//2, boundary = 'trim').sum()

<xarray.Dataset> Size: 112B
Dimensions:  (lon: 2, lat: 2, time: 2)
Coordinates:
  * lon      (lon) float64 16B 1.5 3.5
  * lat      (lat) float64 16B 1.5 3.5
  * time     (time) int64 16B 1 2
Data variables:
    temp     (lon, lat, time) float64 64B 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0